In [1]:
using PyPlot

@everywhere using Distributions
@everywhere include("utils.jl")
@everywhere include("ode.jl")
@everywhere srand(42);

In [18]:
@everywhere function noop(W,X) 
end

@everywhere function SequentialMonteCarlo(X₀, p, γ; μ=0.1, progress=noop)
    # initialization
    N,d = size(X₀)
    w = MvNormal(μ^2 * eye(d))

    X = X₀
    W = ones(N)/N
    Wₙ = SharedArray{Float64}(N)
    
    progress(W,X)
    
    # sampling
    for n = 1:p
        X += rand(w, N)'
        
        # resampling
        if 1 / vecdot(W,W) < N/2
            X = X[rand(Categorical(W), N), :]
            W = ones(N)/N
        end

        @sync @parallel for i=1:N
            Wₙ[i] = γ(n, X[i,:])
        end
        
        W = W.*Wₙ / vecdot(W, Wₙ)
        progress(W,X)
    end
    
    W,X
end

In [6]:
@everywhere function simulate(n, g, b; Θ₀ = 5*pi/180, l = 7.4, τ = 0.01)
    n_times = 0
    times = zeros(n)

    t = 0
    Θ = [Θ₀, 0]
    
    function dΘ(Θ)
        [Θ[2], -(Θ[2]*b + g*sin(Θ[1])/l)]
    end
    
    while n_times < n
        Θₙ = rk4(dΘ, Θ, τ)
        
        # if the angle has changed sign, we linearly approximate the intersection
        # with 0
        if Θ[1] * Θₙ[1] <= 0
            t_intersect = t - τ*Θ[1]/(Θₙ[1] - Θ[1])
            n_times += 1
            times[n_times] = t_intersect
        end
        
        Θ = Θₙ
        t += τ
    end
    
    times
end

In [5]:
@everywhere measurements = open(readdlm,"../data/original_data")
@everywhere measurements_mask = abs(sign(measurements))

@everywhere n_measurements = size(measurements)[1]
@everywhere max_intersects = size(measurements)[2]

prior = dunif(5, 15)

likelihood = m ->
    let
        d = dnorm(0, 1)
        t = simulate(max_intersects, m[1], 0.4, τ=0.1)
        exp(sumoveri(i -> sum(log.(d.(measurements_mask[i,:] .* (measurements[i,:] - t)))), n_measurements))
    end

unnormalized_posterior = m ->
    let
        p = prior(m[1])
        p > 0 ? p*likelihood(m) : 0
    end

#samples, n_accepted = MetropolisHastings(10000, unnormalized_posterior, 0.1, rand(1)*10 + 5)

(::#72) (generic function with 1 method)

In [52]:
@everywhere τs = [1 0.5 0.1 0.05 0.01]

likelihood₁ = (n, m) ->
    let
        d = dnorm(0, 1)
        t = simulate(max_intersects, m[1], 0.4, τ=τs[n])    
        exp(sumoveri(i -> sum(log.(d.(measurements_mask[i,:] .* (measurements[i,:] - t)))), n_measurements))
    end

W,X = SequentialMonteCarlo(rand(1000,1)*10+5, length(τs), likelihood₁);

In [54]:
pygui(true)
ion()

progress = d -> (W, X) ->
    let
        Y = round.(X/d)
        mini = minimum(Y)
        
        n = convert(Int32,maximum(Y) - mini + 1)
        ws = zeros(n)
        
        for i=1:length(W)
            ws[convert(Int32,Y[i]-mini+1)]+=W[i]
        end

        maxW = maximum(ws)
        g = vecdot(W,X)    

        clf()
        title("$( g )")
        ylim(0,maxW*1.1)
        xlim(5,15)
        
        bar((collect(1:n) + mini - 1)*d, ws, width=d)
    
        plot([g; g], [0; maxW*1.1], c="red")
        sleep(0.5)   
    end

W,X = SequentialMonteCarlo(rand(5000,1)*10+5, length(τs), likelihood₁, progress=progress(0.1));